In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.

# Python native
import pickle
from statistics import median as median
from tqdm import tqdm
import os
os.chdir("c:\\Users\\Tim\\Development\\OCELFeatureExtractionExperiments")
from copy import deepcopy

# Data handling
import pandas as pd
import numpy as np

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# PyG
import torch
from ocpa_PyG_integration.PyG_dataset import EventGraphDataset


c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.1+cpu
Cuda available: False
Torch geometric version: 2.2.0


In [3]:
STORAGE_PATH = "data/ocpa-processed/raw/"
SAMPLE_SIZE = 9999  # Can be either 'False' or an integer
with open(f"{STORAGE_PATH}BPI2017-feature_storage(take_sample={SAMPLE_SIZE}).pkl", "rb") as file:
    fs = pickle.load(file)
len(fs.feature_graphs)

31509

In [2]:
STORAGE_PATH = "data/ocpa-processed/raw/"
SAMPLE_SIZE = False  # Can be either 'False' or an integer
with open(f"{STORAGE_PATH}BPI2017-feature_storage(take_sample={SAMPLE_SIZE}).pkl", "rb") as file:
    fs = pickle.load(file)
len(fs.feature_graphs)

31509

In [5]:
egds9999 = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage(take_sample=9999).pkl",
    label_key=("event_remaining_time", ()),
    train=True,
)
egds9999.get_summary()

Processing...
22056it [01:12, 302.98it/s]
Done!
100%|██████████| 22056/22056 [04:15<00:00, 86.43it/s] 


EventGraphDataset (#graphs=22056):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.6 |      4.5 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     41   |     54   |
+------------+----------+----------+

In [2]:
def count_parameters(model) ->int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
ds_train = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage(take_sample=False).pkl",
    label_key=("event_remaining_time", ()),
    train=True,
)

Processing...
22056it [01:14, 294.97it/s]
Done!


In [4]:
ds_test = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage(take_sample=False).pkl",
    label_key=("event_remaining_time", ()),
    test=True,
)

Processing...
9453it [00:13, 693.15it/s]
Done!


In [7]:
ds_train.get_summary()

100%|██████████| 22056/22056 [04:33<00:00, 80.68it/s] 


EventGraphDataset (#graphs=22056):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.6 |      4.5 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     41   |     54   |
+------------+----------+----------+

In [6]:
from model import GCN
import torch 
from torch_geometric.loader import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from tqdm import tqdm


# Initialize model
model = GCN(ds_train.num_node_features, 12)
print(model)
print(f'Number of parameters: {count_parameters(model)}')


# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# data = ds_train.to(device)

# Initialize Optimizer
learning_rate = 0.01
decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=decay)
NUM_GRAPHS_PER_BATCH = 512
# Define loss function (CrossEntropyLoss for Classification Problems with 
# probability distributions)
criterion = torch.nn.MSELoss()

train_loader = DataLoader(ds_train,batch_size=NUM_GRAPHS_PER_BATCH,shuffle=True)
test_loader = DataLoader(ds_test,batch_size=NUM_GRAPHS_PER_BATCH,shuffle=True)

def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n MAE: \n {mean_absolute_error(y_pred, y_true)}")
    print(f"MSE: {mean_squared_error(y_true, y_pred)}")
    print(f"R^2: {r2_score(y_true, y_pred)}")

I'm updated!
GCN(
  (conv1): GCNConv(23, 12)
  (conv2): GCNConv(12, 12)
  (out): Linear(in_features=12, out_features=1, bias=True)
)
Number of parameters: 457


In [7]:


def train(epoch):
      all_preds = []
      all_labels = []

      model.train()
      optimizer.zero_grad() 
      # Use all data as input, because all nodes have node features
      out = model(data.x, data.edge_index)  
      # Only use nodes with labels available for loss calculation --> mask
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  
      loss.backward() 
      optimizer.step()
      return loss

def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        batch.to(device)  
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model(batch.x.float(), 
                                batch.edge_index, 
                                batch.batch) 
        # Calculating the loss and gradients
        loss = loss_fn(torch.squeeze(pred), batch.y.float())
        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
        print(f'{running_loss} \r', end=" ")
        print(f'{step} \r', end=" ")
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return running_loss/step

def test(epoch, model, test_loader, loss_fn):
    all_preds = []
    all_preds_raw = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.to(device)  
        pred = model(batch.x.float(), 
                        batch.edge_index, 
                        batch.batch) 
        loss = loss_fn(torch.squeeze(pred), batch.y.float())

         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    print(all_preds_raw[0][:10])
    print(all_preds[:10])
    print(all_labels[:10])
    calculate_metrics(all_preds, all_labels, epoch, "test")
    return running_loss/step

def run_training(loss_fn):
    
    # Start training
    best_loss = 1000
    early_stopping_counter = 0
    for epoch in range(300): 
        if early_stopping_counter <= 10: # = x * 5 
            # Training
            model.train()
            loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
            print(f"Epoch {epoch} | Train Loss {loss}")

            # Testing
            model.eval()
            if epoch % 5 == 0:
                loss = test(epoch, model, test_loader, loss_fn)
                print(f"Epoch {epoch} | Test Loss {loss}")
                
                # Update best loss
                if float(loss) < best_loss:
                    best_loss = loss
                    # # Save the currently best model 
                    # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1

        else:
            print("Early stopping due to no improvement.")
            return [best_loss]

In [9]:
torch.save(model.state_dict(), 'models/GCN2_adams_network_state_dict.pt')
torch.save(model.state_dict(), 'models/GCN2_adams_network.pt')

In [8]:
results = run_training(criterion)

  2%|▏         | 1/44 [00:00<00:25,  1.71it/s]

 1 980560779571533 

  5%|▍         | 2/44 [00:00<00:18,  2.27it/s]

 2 11311960220337 

  7%|▋         | 3/44 [00:01<00:16,  2.51it/s]

...

 95%|█████████▌| 42/44 [00:17<00:00,  2.74it/s]

 42 95782524347305 

100%|██████████| 44/44 [00:17<00:00,  2.49it/s]

 44 891286820173264 
 MAE: 
 0.8500240445137024
MSE: 0.98283451795578
R^2: 0.017165377658137082
Epoch 95 | Train Loss 0.6111656095493924


[[0.6027333 ]
 [0.38980696]
 [0.38980302]
 [0.36050302]
 [0.6390478 ]
 [0.26745054]
 [0.26469028]
 [0.64010596]
 [0.5943025 ]
 [0.58172464]]
[1. 0. 0. 0. 1. 0. 0. 1. 1. 1.]
[-0.42717418 -0.8441699  -0.8443075  -0.9302815  -0.42668125 -0.98930985
 -0.98930985 -0.42717415 -0.42704135 -0.4271665 ]

 MAE: 
 0.8529074192047119
MSE: 0.9814555048942566
R^2: -0.00758630158563145
Epoch 95 | Test Loss 0.5912032158751237
Early stopping due to no improvement.


In [13]:
state_dict = model.state_dict()
state_dict["out.weight"]

tensor([[ 6.0141e-01, -4.3788e-01,  1.7691e-15, -4.9253e-01,  8.7048e-01,
          1.4169e-02,  9.9122e-01, -3.4069e-05,  4.1620e-01,  7.2336e-01,
          3.9588e-01,  3.8909e-07]])